In [1]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql import SparkSession

In [2]:
builder = (
    SparkSession
    .builder
    .master("spark://spark-master:7077")
    .appName("rawApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
spark.sql("DESCRIBE HISTORY delta.`/opt/workspace/data/raw/iris/`").show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      5|2023-05-27 04:25:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          4|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      4|2023-05-27 04:16:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          3|  Serializable|        false|{numFiles -> 1, n...|        null|Apache-Spark/3.3....|
|      3|2

In [4]:
df = (
    spark
    .read
    .format('delta')
    .option("readChangeFeed", "true")
    .option("startingVersion", 5)
    .load('/opt/workspace/data/raw/iris/')
)

In [5]:
df.show()

AnalysisException: Error getting change data for range [5 , 5] as change data was not
recorded for version [5]. If you've enabled change data feed on this table,
use `DESCRIBE HISTORY` to see when it was first enabled.
Otherwise, to start recording change data, use `ALTER TABLE table_name SET TBLPROPERTIES
(delta.enableChangeDataFeed=true)`.

In [30]:
df = df.drop_duplicates()

In [31]:
df.write.mode('overwrite').format('delta').save('/opt/workspace/data/cleaned/iris/')

In [32]:
spark.stop()